In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs

from scsavailability import features as feat, model as md, plotting as pt

In [ ]:
path = '../data/'

In [ ]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
fa = pd.read_csv(path + 'scs_tote_matched.csv')

In [ ]:
at = feat.pre_process_AT(at)

In [ ]:
av = feat.pre_process_av(av)

In [ ]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

In [ ]:
lu = pd.read_csv('../data/module_lookup.csv')

In [ ]:
# Copy desk
fa['Desk_edit'] = fa['Desk']
# Mark SCSs
fa.loc[fa['PLC'].str.contains('SCS'), 'Desk_edit'] = fa.loc[fa['PLC'].str.contains('SCS'), 'PLC']
# Mark PTTs
#fa.loc[fa['Alert'].str.contains('PTT'), 'Desk_edit'] = 'PTT'
# Set NA desk for outside stuff
fa.loc[fa['PLC'].isin(['C23', 'C16', 'C15', 'C17']), 'Desk_edit'] = 'X'
fa['PLCN'] = fa['PLC'].str.extract('((?<=C)[0-9]{2})')[0].astype('float')
fa.loc[fa['PLCN'] > 34, 'Desk_edit'] = 'X'

In [ ]:
fa.shape

In [ ]:
aa = pd.merge(fa, lu, how='left', on=['PLC', 'Desk_edit'])

In [ ]:
aa[~(aa['Module'] == aa['MODULE'])]#[['Module', 'MODULE']]

In [ ]:
fa['Alert'].str.replace('([0-9]+)|[:_]', '', regex=True)

In [ ]:
fa['extract'] = fa['Alert'].str.extract('((?<=\s)([A-Z]*[a-z|\s|:-\(]+(?![A-Z][0-9]{2})[0-9]*)+)')[0].str.strip()

In [ ]:
a = fa.groupby(['Fault ID', 'extract'])['PLC'].apply(lambda x: list(pd.Series((' '.join(x)).split()).drop_duplicates())).reset_index()

In [ ]:
multi_desc = pd.Series(a['extract'].value_counts()[a['extract'].value_counts() > 1].index)

In [ ]:
df_plc = df.groupby(['Fault ID', 'Description'])['PLC'].apply(lambda x: list(pd.Series((' '.join(x)).split()).drop_duplicates())).reset_index()

multi_desc = pd.Series(a['extract'].value_counts()[a['extract'].value_counts() > 1].index)
a['Description'] = a['extract']
a.loc[a['extract'].isin(multi_desc), 'Description'] = (a.loc[a['extract'].isin(multi_desc), 'extract'] + ' ' +
                                                       a.loc[a['extract'].isin(multi_desc), 'PLC'].astype('str'))

In [ ]:
a

In [ ]:
a['Fault ID'].value_counts()

In [ ]:
a['Description'].value_counts()

In [ ]:
a[a['extract']=='fire alarm']

In [ ]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

In [ ]:
fa_sel = feat.fault_select(fa_floor, select_level = 'Tote Colour', selection = ['Blue','Both'])

fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level='code', agg_type = 'count')

In [ ]:
av,at = feat.av_at_select(av, at, remove_high_AT = False)

av_agg = feat.aggregate_availability(av, agg_level = 'Module')
at_agg = feat.aggregate_totes(at, agg_level = 'Module')

In [ ]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, target = 'Blue Tote Loss',faults=True, totes = True, agg_level = 'Module')

In [ ]:
df = df[df['TOTES']<60].reset_index(drop=True)

In [ ]:
X,y = md.gen_feat_var(df)

In [ ]:
X_train, X_test, y_train, y_test = md.split(X,y,test_size=0.3,random_state=101)

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,num_trees=100, criterion = 'mse', max_depth=None, dtree=False)

#X_sel_rf = md.select_features(X, X_train, y_train, RF_mdl, thres = np.inf(), max_feat=1000)

Linear_mdl,predictions_LM=md.run_LR_model(X_train, X_test, y_train, y_test)

#X_sel_lm = md.select_features(X, X_train, y_train, Linear_mdl, thres = np.inf(), max_feat=1000)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)

# cv_R2 = md.cross_validate_r2(RF_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)